In [2]:
# !pip install sympy

In [1]:
from sympy import symbols, cos, sin, Matrix, diag, zeros, eye
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

# Symbolic functions

In [2]:
def skew(v):
    return Matrix([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])

In [3]:
def exp3(omega, theta):
    omega = skew(omega)
    R = eye(3) + sin(theta) * omega + (1 - cos(theta)) * omega * omega
    return R

In [4]:
def exp3z(z, theta):
    R = eye(3) + sin(theta) * z + (1 - cos(theta)) * z * z
    return R

In [5]:
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3, :3] = exp3(twist[:3], theta)
    T[:3, 3] = (eye(3) * theta + (1 - cos(theta)) * omega +
                (theta - sin(theta)) * omega * omega) * v
    return T

In [6]:
def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

# 5.1.1 Space Jacobian
## Example: Scara Robot (RRRP)


### Define symbols

In [9]:
th1, th2, th3, th4, th5, th6, c1, c2, c3, c4, s1, s2, s3, s4 = dynamicsymbols("theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, c1, c2, c3, c4, s1, s2, s3, s4")
L = symbols("L")

In [11]:
w1 = Matrix([[0], [0], [1]])
w2 = Matrix([[0], [1], [0]])
w3 = Matrix([[1], [0], [0]])
w4 = Matrix([[1], [0], [0]])
w5 = Matrix([[1], [0], [0]])
w6 = Matrix([[0], [1], [0]])

q1 = Matrix([[0], [0], [0]])
q2 = Matrix([[0], [0], [0]])
q3 = Matrix([[0], [0], [0]])

ws1 = w1
vs1 = Matrix([[0], [0], [0]])

ws2 = exp3(w1, th1)*w2
vs2 = Matrix([[0], [0], [0]])

ws3 = exp3(w1, th1)*exp3(w2, th2)*w3
vs3 = Matrix([[0], [0], [0]])

ws4 = exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*w4
q4 = exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*Matrix([[0],[L],[0]])
vs4 = -skew(ws4)*q4

ws5 = exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*exp3(-w4, th4)*w5
q5 = q4 + exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*exp3(-w4, th4)*Matrix([[0],[L],[0]])
vs5 = -skew(ws5)*q5

ws6= exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*exp3(-w4, th4)*exp3(-w5, th5)*w6
q6 = q5 + exp3(w1, th1)*exp3(w2, th2)*exp3(-w3, th3)*exp3(-w4, th4)*exp3(-w5, th5)*Matrix([[0],[L],[0]])
vs6 = -skew(ws6)*q6


ws1.simplify()
ws2.simplify()
ws3.simplify()
ws4.simplify()
ws5.simplify()
ws6.simplify()
vs1.simplify()
vs2.simplify()
vs3.simplify()
vs4.simplify()
vs5.simplify()
vs6.simplify()
ws6

⎡-sin(θ₃ + θ₄ + θ₅)⋅sin(θ₂)⋅cos(θ₁) - sin(θ₁)⋅cos(θ₃ + θ₄ + θ₅)⎤
⎢                                                              ⎥
⎢-sin(θ₃ + θ₄ + θ₅)⋅sin(θ₁)⋅sin(θ₂) + cos(θ₃ + θ₄ + θ₅)⋅cos(θ₁)⎥
⎢                                                              ⎥
⎣                  -sin(θ₃ + θ₄ + θ₅)⋅cos(θ₂)                  ⎦

In [12]:
Js = zeros(6,6)
Js

⎡0  0  0  0  0  0⎤
⎢                ⎥
⎢0  0  0  0  0  0⎥
⎢                ⎥
⎢0  0  0  0  0  0⎥
⎢                ⎥
⎢0  0  0  0  0  0⎥
⎢                ⎥
⎢0  0  0  0  0  0⎥
⎢                ⎥
⎣0  0  0  0  0  0⎦

In [14]:
Js[:3,0] = ws1
Js[:3,1] = ws2
Js[:3,2] = ws3
Js[:3,3] = ws4
Js[:3,4] = ws5
Js[:3,5] = ws6
Js[3:,0] = vs1
Js[3:,1] = vs2
Js[3:,2] = vs3
Js[3:,3] = vs4
Js[3:,4] = vs5
Js[3:,5] = vs6
Js

⎡0  -sin(θ₁)  cos(θ₁)⋅cos(θ₂)                 cos(θ₁)⋅cos(θ₂)                 
⎢                                                                             
⎢0  cos(θ₁)   sin(θ₁)⋅cos(θ₂)                 sin(θ₁)⋅cos(θ₂)                 
⎢                                                                             
⎢1     0         -sin(θ₂)                         -sin(θ₂)                    
⎢                                                                             
⎢0     0             0         L⋅(sin(θ₁)⋅sin(θ₃) - sin(θ₂)⋅cos(θ₁)⋅cos(θ₃))  
⎢                                                                             
⎢0     0             0         -L⋅(sin(θ₁)⋅sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₁)) 
⎢                                                                             
⎣0     0             0                       -L⋅cos(θ₂)⋅cos(θ₃)               

                                           cos(θ₁)⋅cos(θ₂)                    
                                                   

In [18]:
Js[:,0]

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢1⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [19]:
Js[:,1]

⎡-sin(θ₁)⎤
⎢        ⎥
⎢cos(θ₁) ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎣   0    ⎦

In [20]:
Js[:,2]

⎡cos(θ₁)⋅cos(θ₂)⎤
⎢               ⎥
⎢sin(θ₁)⋅cos(θ₂)⎥
⎢               ⎥
⎢   -sin(θ₂)    ⎥
⎢               ⎥
⎢       0       ⎥
⎢               ⎥
⎢       0       ⎥
⎢               ⎥
⎣       0       ⎦

In [21]:
Js[:,3]

⎡               cos(θ₁)⋅cos(θ₂)                ⎤
⎢                                              ⎥
⎢               sin(θ₁)⋅cos(θ₂)                ⎥
⎢                                              ⎥
⎢                   -sin(θ₂)                   ⎥
⎢                                              ⎥
⎢L⋅(sin(θ₁)⋅sin(θ₃) - sin(θ₂)⋅cos(θ₁)⋅cos(θ₃)) ⎥
⎢                                              ⎥
⎢-L⋅(sin(θ₁)⋅sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₁))⎥
⎢                                              ⎥
⎣              -L⋅cos(θ₂)⋅cos(θ₃)              ⎦

In [24]:
Js[:,4]

⎡                                          cos(θ₁)⋅cos(θ₂)                    
⎢                                                                             
⎢                                          sin(θ₁)⋅cos(θ₂)                    
⎢                                                                             
⎢                                              -sin(θ₂)                       
⎢                                                                             
⎢L⋅(sin(θ₃ + θ₄)⋅sin(θ₁) + sin(θ₁)⋅sin(θ₃) - sin(θ₂)⋅cos(θ₃ + θ₄)⋅cos(θ₁) - si
⎢                                                                             
⎢-L⋅(sin(θ₃ + θ₄)⋅cos(θ₁) + sin(θ₁)⋅sin(θ₂)⋅cos(θ₃ + θ₄) + sin(θ₁)⋅sin(θ₂)⋅cos
⎢                                                                             
⎣                                -L⋅(cos(θ₃ + θ₄) + cos(θ₃))⋅cos(θ₂)          

                       ⎤
                       ⎥
                       ⎥
                       ⎥
                       ⎥
     

In [25]:
Js[:,5]

⎡-sin(θ₃ + θ₄ + θ₅)⋅sin(θ₂)⋅cos(θ₁) - sin(θ₁)⋅cos(θ₃ + θ₄ + θ₅)⎤
⎢                                                              ⎥
⎢-sin(θ₃ + θ₄ + θ₅)⋅sin(θ₁)⋅sin(θ₂) + cos(θ₃ + θ₄ + θ₅)⋅cos(θ₁)⎥
⎢                                                              ⎥
⎢                  -sin(θ₃ + θ₄ + θ₅)⋅cos(θ₂)                  ⎥
⎢                                                              ⎥
⎢         -L⋅(sin(θ₄ + θ₅) + sin(θ₅))⋅cos(θ₁)⋅cos(θ₂)          ⎥
⎢                                                              ⎥
⎢         -L⋅(sin(θ₄ + θ₅) + sin(θ₅))⋅sin(θ₁)⋅cos(θ₂)          ⎥
⎢                                                              ⎥
⎣              L⋅(sin(θ₄ + θ₅) + sin(θ₅))⋅sin(θ₂)              ⎦